In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/12월/나무문화_12월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

392


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_12월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,흑흑 벌써 1년이 지났다니ㅠㅠ 주화의 2022년 요약 시작 ㅎㅅㅎ 2022 01 어...,0,"['흑', '흑', '벌써', '1년', '이', '지났다니', 'ㅠㅠ', '주화'...","[('흑', 'Adverb'), ('흑', 'Adverb'), ('벌써', 'Nou..."
1,웃겨서 찍음,0,"['웃겨서', '찍음']","[('웃기다', 'Verb'), ('찍다', 'Verb')]"
2,그리고 김시연때뭉네 새해부터 부산감 광안리도 갔는데 횡단보도 기다리는 강아쥐 넘 귀...,0,"['그리고', '김시연', '때뭉', '네', '새해', '부터', '부산', '감...","[('그리고', 'Conjunction'), ('김시연', 'Noun'), ('때뭉..."
3,모범 강아쥐 그리고 바로 담날 최성빈이 경주오겠다고 놀아달라고 놀아달라고 해서 놀아줌ㅋ,0,"['모범', '강', '아', '쥐', '그리고', '바로', '담날', '최성빈'...","[('모범', 'Noun'), ('강', 'Noun'), ('아', 'Josa'),..."
4,녹차라떼 거하게 쏟을 뻔 한거 최성빈이 잡아서 다행히 저만큼만 쏟음,0,"['녹차', '라떼', '거', '하게', '쏟을', '뻔', '한', '거', '...","[('녹차', 'Noun'), ('라떼', 'Noun'), ('거', 'Noun')..."
...,...,...,...,...
1011108,돼지의 모습이 신기한지 우리 아이들 이곳에서 한참을 봤네요 ㅎㅎㅎ,20247,"['돼지', '의', '모습', '이', '신기한지', '우리', '아이', '들'...","[('돼지', 'Noun'), ('의', 'Josa'), ('모습', 'Noun')..."
1011109,남문 주변에는 닭 말 개 등 동물도 가까이서 볼 수 있어요 마을내에는 전통 민박에서...,20247,"['남문', '주변', '에는', '닭', '말', '개', '등', '동물', '...","[('남문', 'Noun'), ('주변', 'Noun'), ('에는', 'Josa'..."
1011110,문화유산과 공존하며 살아가는 마을 주민들의 삶을 느껴볼 수도 있다고 해요,20247,"['문화유산', '과', '공존', '하며', '살아가는', '마을', '주민', ...","[('문화유산', 'Noun'), ('과', 'Josa'), ('공존', 'Noun..."
1011111,제주 성읍민속마을 맛집 성읍치킨튀에서 맛있게 먹고 마을투어 해보는 것을 추천해요 옛...,20247,"['제주', '성', '읍민', '속', '마을', '맛집', '성', '읍', '...","[('제주', 'Noun'), ('성', 'Modifier'), ('읍민', 'No..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,흑흑 벌써 1년이 지났다니ㅠㅠ 주화의 2022년 요약 시작 ㅎㅅㅎ 2022 01 어...,0,"['흑', '흑', '벌써', '1년', '이', '지났다니', 'ㅠㅠ', '주화'...","[('흑', 'Adverb'), ('흑', 'Adverb'), ('벌써', 'Nou...",흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
1,웃겨서 찍음,0,"['웃겨서', '찍음']","[('웃기다', 'Verb'), ('찍다', 'Verb')]",웃겨서 찍음
2,그리고 김시연때뭉네 새해부터 부산감 광안리도 갔는데 횡단보도 기다리는 강아쥐 넘 귀...,0,"['그리고', '김시연', '때뭉', '네', '새해', '부터', '부산', '감...","[('그리고', 'Conjunction'), ('김시연', 'Noun'), ('때뭉...",그리고 김시연 때뭉 네 새해 부터 부산 감 광안 리도 갔는데 횡단보도 기다리는 강 ...
3,모범 강아쥐 그리고 바로 담날 최성빈이 경주오겠다고 놀아달라고 놀아달라고 해서 놀아줌ㅋ,0,"['모범', '강', '아', '쥐', '그리고', '바로', '담날', '최성빈'...","[('모범', 'Noun'), ('강', 'Noun'), ('아', 'Josa'),...",모범 강 아 쥐 그리고 바로 담날 최성빈 이 경주 오겠다고 놀아 달라 고 놀아 달라...
4,녹차라떼 거하게 쏟을 뻔 한거 최성빈이 잡아서 다행히 저만큼만 쏟음,0,"['녹차', '라떼', '거', '하게', '쏟을', '뻔', '한', '거', '...","[('녹차', 'Noun'), ('라떼', 'Noun'), ('거', 'Noun')...",녹차 라떼 거 하게 쏟을 뻔 한 거 최성빈 이 잡아서 다행히 저만큼 만 쏟음
...,...,...,...,...,...
1011108,돼지의 모습이 신기한지 우리 아이들 이곳에서 한참을 봤네요 ㅎㅎㅎ,20247,"['돼지', '의', '모습', '이', '신기한지', '우리', '아이', '들'...","[('돼지', 'Noun'), ('의', 'Josa'), ('모습', 'Noun')...",돼지 의 모습 이 신기한지 우리 아이 들 이 곳 에서 한참 을 봤네요 ㅎㅎㅎ
1011109,남문 주변에는 닭 말 개 등 동물도 가까이서 볼 수 있어요 마을내에는 전통 민박에서...,20247,"['남문', '주변', '에는', '닭', '말', '개', '등', '동물', '...","[('남문', 'Noun'), ('주변', 'Noun'), ('에는', 'Josa'...",남문 주변 에는 닭 말 개 등 동물 도 가까이 서 볼 수 있어요 마을 내 에는 전통...
1011110,문화유산과 공존하며 살아가는 마을 주민들의 삶을 느껴볼 수도 있다고 해요,20247,"['문화유산', '과', '공존', '하며', '살아가는', '마을', '주민', ...","[('문화유산', 'Noun'), ('과', 'Josa'), ('공존', 'Noun...",문화유산 과 공존 하며 살아가는 마을 주민 들 의 삶 을 느껴 볼 수도 있다고 해 요
1011111,제주 성읍민속마을 맛집 성읍치킨튀에서 맛있게 먹고 마을투어 해보는 것을 추천해요 옛...,20247,"['제주', '성', '읍민', '속', '마을', '맛집', '성', '읍', '...","[('제주', 'Noun'), ('성', 'Modifier'), ('읍민', 'No...",제주 성 읍민 속 마을 맛집 성 읍 치킨 튀에서 맛있게 먹고 마을 투어 해보는 것 ...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,흑흑 벌써 1년이 지났다니ㅠㅠ 주화의 2022년 요약 시작 ㅎㅅㅎ 2022 01 어...,0,"['흑', '흑', '벌써', '1년', '이', '지났다니', 'ㅠㅠ', '주화'...","[('흑', 'Adverb'), ('흑', 'Adverb'), ('벌써', 'Nou...",흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
1,웃겨서 찍음,0,"['웃겨서', '찍음']","[('웃기다', 'Verb'), ('찍다', 'Verb')]",웃겨서 찍음,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
2,그리고 김시연때뭉네 새해부터 부산감 광안리도 갔는데 횡단보도 기다리는 강아쥐 넘 귀...,0,"['그리고', '김시연', '때뭉', '네', '새해', '부터', '부산', '감...","[('그리고', 'Conjunction'), ('김시연', 'Noun'), ('때뭉...",그리고 김시연 때뭉 네 새해 부터 부산 감 광안 리도 갔는데 횡단보도 기다리는 강 ...,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
3,모범 강아쥐 그리고 바로 담날 최성빈이 경주오겠다고 놀아달라고 놀아달라고 해서 놀아줌ㅋ,0,"['모범', '강', '아', '쥐', '그리고', '바로', '담날', '최성빈'...","[('모범', 'Noun'), ('강', 'Noun'), ('아', 'Josa'),...",모범 강 아 쥐 그리고 바로 담날 최성빈 이 경주 오겠다고 놀아 달라 고 놀아 달라...,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
4,녹차라떼 거하게 쏟을 뻔 한거 최성빈이 잡아서 다행히 저만큼만 쏟음,0,"['녹차', '라떼', '거', '하게', '쏟을', '뻔', '한', '거', '...","[('녹차', 'Noun'), ('라떼', 'Noun'), ('거', 'Noun')...",녹차 라떼 거 하게 쏟을 뻔 한 거 최성빈 이 잡아서 다행히 저만큼 만 쏟음,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
...,...,...,...,...,...,...
1011108,돼지의 모습이 신기한지 우리 아이들 이곳에서 한참을 봤네요 ㅎㅎㅎ,20247,"['돼지', '의', '모습', '이', '신기한지', '우리', '아이', '들'...","[('돼지', 'Noun'), ('의', 'Josa'), ('모습', 'Noun')...",돼지 의 모습 이 신기한지 우리 아이 들 이 곳 에서 한참 을 봤네요 ㅎㅎㅎ,얼마 전 결혼기념일 에 표선 독채 숙소 에 다녀오면서 제주 초가집 치킨 가게 를 들...
1011109,남문 주변에는 닭 말 개 등 동물도 가까이서 볼 수 있어요 마을내에는 전통 민박에서...,20247,"['남문', '주변', '에는', '닭', '말', '개', '등', '동물', '...","[('남문', 'Noun'), ('주변', 'Noun'), ('에는', 'Josa'...",남문 주변 에는 닭 말 개 등 동물 도 가까이 서 볼 수 있어요 마을 내 에는 전통...,얼마 전 결혼기념일 에 표선 독채 숙소 에 다녀오면서 제주 초가집 치킨 가게 를 들...
1011110,문화유산과 공존하며 살아가는 마을 주민들의 삶을 느껴볼 수도 있다고 해요,20247,"['문화유산', '과', '공존', '하며', '살아가는', '마을', '주민', ...","[('문화유산', 'Noun'), ('과', 'Josa'), ('공존', 'Noun...",문화유산 과 공존 하며 살아가는 마을 주민 들 의 삶 을 느껴 볼 수도 있다고 해 요,얼마 전 결혼기념일 에 표선 독채 숙소 에 다녀오면서 제주 초가집 치킨 가게 를 들...
1011111,제주 성읍민속마을 맛집 성읍치킨튀에서 맛있게 먹고 마을투어 해보는 것을 추천해요 옛...,20247,"['제주', '성', '읍민', '속', '마을', '맛집', '성', '읍', '...","[('제주', 'Noun'), ('성', 'Modifier'), ('읍민', 'No...",제주 성 읍민 속 마을 맛집 성 읍 치킨 튀에서 맛있게 먹고 마을 투어 해보는 것 ...,얼마 전 결혼기념일 에 표선 독채 숙소 에 다녀오면서 제주 초가집 치킨 가게 를 들...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,흑 흑 벌써 1년 이 지났다니 ㅠㅠ 주화 의 2022년 요약 시작 ㅎㅅㅎ 2022 ...
1,1,배리 랑 연남동 나들이 애견 동반 이 되는 히 포 카페테리아 라는 곳 으로 점심 을...
2,2,노래 여 노래 여 이근배 푸른 에 서서 피 묻은 의 가슴 을 씻는 내 가난한 꽃 은...
3,3,어느덧 12월 31일 입니다 요 번 달 은 꽤 나 감성 적 이니까 마루 부리 체 로...
4,4,광화문 교보 문고 에서 책 을 읽다가 밤 8시 쯤 나왔을 때 였다 저 멀리 뭔가 빛...
...,...,...
20242,20243,꼭 가보고 싶었던 빅 테크 기업 NAVER 지나면서 항상 부러 움 경 외 의 눈 으...
20243,20244,서울 관악 가볼만 한 곳 대학동 생활 상권 영수증 투어 재밌어 대학동 영수증 투어 ...
20244,20245,은은한 갈대 의 담백한 정취 겨울 이 더디 오는 11월 말 낮 에는 아직 나들이 하...
20245,20246,원래 이 전시 를 보러 국현 에 간 게 아니었다 다른 걸 보러 갔는데 막상 보니 내...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(20247, 474224)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01319149, 0.0369893 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,474214,474215,474216,474217,474218,474219,474220,474221,474222,474223
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20242,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20243,0.000000,0.012507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20244,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20245,0.013191,0.036989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.035228,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,000100,000117207,00012,000120,0001247843,00015,...,힛팅건,힛포,힜다,힝,힝구,힝둥새,힝행,힝힝,힠,힣
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20242,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20243,0.000000,0.012507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20244,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20245,0.013191,0.036989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.035228,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/12월/나무문화_12월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/12월/나무문화_12월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/12월/나무문화_12월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')